calitatea localizarii corecte a textului in imagine

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time
import os
from shapely.geometry import Polygon

os.environ["VISION_KEY"] ="Your_key"
os.environ["VISION_ENDPOINT"] ="https://berindeiepaulmihai.cognitiveservices.azure.com/"
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [2]:
def get_bounding_boxes(read_result):
    boxes = []
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                boxes.append(line.bounding_box)
    return boxes


files = os.listdir('img')
folder = 'img'

Box_rez = dict()

for file in files:
    img = open(os.path.join(folder, file), "rb")
    read_response = computervision_client.read_in_stream(
        image=img,
        mode="Printed",
        raw=True
    )

    operation_id = read_response.headers['Operation-Location'].split('/')[-1]
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)
    if file not in Box_rez:
        Box_rez[file] = get_bounding_boxes(read_result)

print(Box_rez)

{'test1.jpg': [[534.0, 261.0, 572.0, 947.0, 501.0, 953.0, 453.0, 261.0], [409.0, 277.0, 517.0, 1362.0, 437.0, 1365.0, 336.0, 283.0]], 'test2.jpg': [[109.0, 98.0, 487.0, 101.0, 485.0, 214.0, 108.0, 211.0], [156.0, 210.0, 443.0, 205.0, 444.0, 315.0, 158.0, 316.0]], 'test3.png': [[221.0, 128.0, 1023.0, 187.0, 1005.0, 429.0, 210.0, 320.0], [230.0, 477.0, 466.0, 480.0, 469.0, 655.0, 233.0, 651.0], [308.0, 797.0, 547.0, 774.0, 562.0, 940.0, 319.0, 956.0]]}


In [3]:
def calculate_proportion(box1, box2):

    polygon1 = Polygon([(box1[0], box1[1]), (box1[2], box1[3]),
                     (box1[4], box1[5]), (box1[6], box1[7])])

    polygon2 = Polygon([(box2[0], box2[1]), (box2[2], box2[3]),
                     (box2[4], box2[5]), (box2[6], box2[7])])

    intersection_area = polygon1.intersection(polygon2).area
    union_area = polygon1.area + polygon2.area - intersection_area
    return intersection_area / union_area if union_area > 0 else 0.0

groundTruth = {
    'test1.jpg': [[534.0, 246.0, 578.0, 945.0, 501.0, 963.0, 457.0, 281.0], [409.0, 273.0, 511.0, 1362.0, 442.0, 1315.0, 346.0, 273.0]],
    'test2.jpg': [[109.0, 100.0, 480.0, 109.0, 475.0, 216.0, 128.0, 212.0], [156.0, 220.0, 444.0, 202.0, 441.0, 345.0, 138.0, 312.0]],
    'test3.png': [[224.0, 125.0, 1033.0, 196.0, 1025.0, 419.0, 220.0, 325.0], [238.0, 475.0, 470.0, 460.0, 463.0, 625.0, 213.0, 691.0], [318.0, 805.0, 577.0, 775.0, 567.0, 941.0, 314.0, 952.0]]
}

for file in Box_rez:
    s = 0
    c = 0
    for box1, box2 in zip(Box_rez[file],groundTruth[file]):
        s = s + calculate_proportion(box1, box2)
        c += 1
    print(f"Pentru poza {file}, proportia este {s/c*100}%")

Pentru poza test1.jpg, proportia este 86.48800054663172%
Pentru poza test2.jpg, proportia este 86.16411124063188%
Pentru poza test3.png, proportia este 87.56563116526715%
